# AKBANK GENAI BOOTCAMP PROJESİ: KAHVE TARİFLERİ RAG CHATBOT'U

**Projenin Amacı:** RAG (Retrieval Augmented Generation) mimarisini kullanarak kahve tarifleri hakkında doğru bilgi sağlayan bir chatbot geliştirmek ve Gradio arayüzü ile sunmaktır.

**Değerlendirme Kriterleri:** Bu Notebook, projenin tüm değerlendirme kriterlerini karşılayacak şekilde (Veri Seti Hazırlama, Çalışma Kılavuzu, Çözüm Mimarisi, Web Arayüzü/Product Kılavuzu) belgelenmiştir.
---

In [ ]:
# Gerekli LangChain, Gemini entegrasyonu, ChromaDB, ve BGE Embeddings kütüphanelerini kuralım
!pip install -q langchain langchain-google-genai chromadb pypdf langchain-community sentence-transformers gradio
print("Gerekli kütüphaneler kuruldu.")

# 🛠️ 3. PROJE ADIMI: KODUNUZUN ÇALIŞMA KILAVUZU

Projenin başarıyla çalıştırılabilmesi için gereken adımlar ve gereksinimler:

1.  **Gereksinimler:** Python 3.10+ ve Colab Notebook ortamı kullanılmalıdır.
2.  **Kütüphane Kurulumu:** Notebook'taki ilk hücrede bulunan `!pip install ...` komutları ile tüm bağımlılıklar kurulur.
3.  **API Anahtarı:** Google Gemini API anahtarı alınmalı ve **'GEMINI_API_KEY'** adıyla Colab Secrets'a (Sırlar) tanımlanmalıdır.
4.  **Çalıştırma:** Notebook'taki hücreler, sırayla ve hatasız bir şekilde çalıştırılmalıdır. İlgili python (veya notebook) dosyanızın çalıştırılması gibi adımlar burada yer alacaktır.

In [ ]:
import os
from google.colab import userdata

# Lütfen Colab'ın sol panelindeki Anahtar (🔒) simgesine tıklayarak
# 'GEMINI_API_KEY' adıyla API anahtarınızı eklediğinizden emin olun.

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    print("API Anahtarı başarıyla ayarlandı.")
except Exception as e:
    print(f"HATA: API anahtarı ayarlanırken sorun oluştu. Detay: {e}")
    print("Lütfen 'GEMINI_API_KEY' adıyla API anahtarınızı Colab Secrets'a ekleyin.")

# Geliştirme için kullanılacak modeller
# Embedding için stabil bir HF modeli, LLM için Gemini kullanacağız.
EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
LLM_MODEL = "gemini-2.5-flash"
print(f"Embedding Model: {EMBEDDING_MODEL}")
print(f"LLM Model: {LLM_MODEL}")

In [ ]:
from langchain_community.document_loaders import TextLoader

FILE_PATH = "kahve_tarifleri.txt"

# Metin dosyasını LangChain'in TextLoader'ı ile okuyun
loader = TextLoader(FILE_PATH)
documents = loader.load()

print(f"Veri dosyası yüklendi. Toplam {len(documents)} adet doküman bulundu.")
print(f"İlk 100 karakter: {documents[0].page_content[:100]}...")

# ☕ 2. PROJE ADIMI: VERİ SETİ HAZIRLAMA

Bu projede kullanılan veri seti, **24 farklı popüler kahve tarifini (Türk Kahvesi, Latte, Mocha, Cold Brew vb.)** ve bunların yapılış adımlarını, malzemelerini içeren **kahve_tarifleri.txt** adlı metin dosyasıdır.

Veri setinizle ilgili genel bilgilerden, içeriğinden ve proje konunuzdan bahsetmeniz yeterlidir. Veri, RAG sisteminin bilgi tabanını oluşturmaktadır.

# 🏗️ 4. PROJE ADIMI: ÇÖZÜM MİMARİNİZ

Bu proje, LangChain çerçevesini kullanan standart bir RAG (Retrieval Augmented Generation) mimarisi ile tasarlanmıştır.

**Projenin Çözdüğü Problem:** Kullanıcıların statik bir dokümandan (kahve tarifleri) aradıkları bilgiyi, karmaşık bir arama yapmadan, doğal dil kullanarak anında doğru bir şekilde almasını sağlamaktır.

**RAG Mimarisi Bileşenleri:**

| RAG Aşaması | Bileşen | Kullanılan Teknoloji | Açıklama |
| :--- | :--- | :--- | :--- |
| **1. Veri Hazırlığı** | Loader & Splitter | `TextLoader` & `RecursiveCharacterTextSplitter` | Veri, hassasiyet için $400$ karakterlik parçalara ayrılır. |
| **2. Gömme (Embedding)** | Embedding Model | HuggingFace BGE Embeddings | Parçalar, arama için sayısal vektörlere dönüştürülür. |
| **3. Depolama & Erişim** | Vector Store & Retriever | ChromaDB & `db.as_retriever()` | Vektörler saklanır. Sorgu geldiğinde, en alakalı $3$ parça (`k=3`) geri getirilir. |
| **4. Üretme (Generation)** | LLM & Prompt | Gemini 2.5 Flash | Geri getirilen bağlam, $\text{Gemini}$'a gönderilerek son yanıt üretilir. |

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RAG hassasiyetini artırmak için küçük parçalar ve çakışma kullanıyoruz
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,       # Daha küçük parça boyutu
    chunk_overlap=100,    # Parçalar arası çakışma
)

# Belgeleri parçalara ayırın
texts = text_splitter.split_documents(documents)

print(f"Toplam bölünmüş parça (chunk) sayısı: {len(texts)}")

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

# BGE Embeddings modelini kullanın (stabil ve Google API'a bağımlı değil)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print("HuggingFace Bge Embeddings modeli yüklendi.")

# ChromaDB veritabanını oluşturun ve veriyi ekleyin.
db = Chroma.from_documents(texts, embeddings, persist_directory="./kahve_chroma_db")

print("ChromaDB Vektör Veritabanı oluşturuldu ve veriler eklendi.")

# Vektör veritabanını bir Geri Getirici'ye (Retriever) dönüştürün.
retriever = db.as_retriever(search_kwargs={"k": 3})

print("Retriever (Geri Getirici) kullanıma hazır.")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

# Gemini LLM Modelini ayarlayın (Generation adımı için)
llm = ChatGoogleGenerativeAI(model=LLM_MODEL, temperature=0.2, api_key=os.environ["GEMINI_API_KEY"])

# RAG için kullanılacak Şablonu (Prompt) tanımlayın
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Sen bir Kahve Tarifleri Uzmanı Chatbot'sun. Yalnızca sağlanan 'Bağlam'daki kahve tariflerini kullanarak kullanıcı sorularını yanıtla. "
     "Eğer tarif bağlamda yoksa, kibarca 'Bu tarifi maalesef bilgi kaynaklarımda bulamadım.' de."
     "\n\nBağlam: {context}"),
    ("human", "{input}"),
])

# Dokümanları birleştiren zinciri oluşturun
document_chain = create_stuff_documents_chain(llm, prompt)

# Geri getirme ve Üretme zincirlerini birleştirin
rag_chain = create_retrieval_chain(retriever, document_chain)

print("RAG Zinciri oluşturuldu. Şimdi test edelim.")

# TEST 1: Bilgi Kaynağında Olan Soru (Latte tarifi)
test_soru_1 = "Latte nasıl yapılır? Adımlarını söyle."
response_1 = rag_chain.invoke({"input": test_soru_1})
print(f"\nSORGU: {test_soru_1}")
print("--------------------------------------------------")
print(f"CEVAP:\n{response_1['answer']}")

In [ ]:
import gradio as gr
from gradio.themes.utils import colors
import os

# Chatbot'un ana işlevini Gradio için bir fonksiyona saralım
def chatbot_response(message, history):
    # 'rag_chain' objesi hafızada olmalıdır.
    try:
        response = rag_chain.invoke({"input": message})
        answer = response["answer"]

        # Cevabın sonuna not ekleme
        return answer + "\n\n*(Bilgiler, kahve tarifleri bilgi kaynağımızdan derlenmiştir.)*"

    except Exception as e:
        return f"Üzgünüm, bir hata oluştu: {e}"

# Gradio arayüzünü oluşturun (Basit ve Temalı Versiyon)
demo = gr.ChatInterface(
    fn=chatbot_response,
    title="☕ Kahve Tarifleri Uzmanı Chatbot ☕",
    textbox=gr.Textbox(placeholder="Örn: Frappe nasıl yapılır?"),
    chatbot=gr.Chatbot(height=400),
    examples=["Latte nasıl yapılır?", "Affogato için malzemeler nelerdir?", "Dibek kahvesi ile Türk kahvesi arasındaki fark nedir?"],

    # Kahve temasına uygun renkler
    theme=gr.themes.Soft(
        primary_hue=colors.orange,
        secondary_hue=colors.blue,
        neutral_hue=colors.gray
    ),
    css="h1 {color: #6F4E37; font-family: 'Georgia', serif;}"
)

# Colab'da çalıştırmak için "share=True"
print("Gradio Arayüzü başlatılıyor. Lütfen çıkan Public URL'yi kullanın.")
demo.queue().launch(share=True)

# 🖥️ 5. PROJE ADIMI: WEB ARAYÜZÜ & PRODUCT KILAVUZU

Projenin canlı demosu, Gradio kullanılarak sunulmuştur. Bu arayüz, projenin kabiliyetlerini doğrudan test etmenizi sağlar.

**Deploy Linkiniz:**
(Lütfen buraya Gradio'dan aldığınız **geçici veya kalıcı** Public URL'yi yapıştırın.)

**Çalışma Akışı ve Test:**
1.  Yukarıdaki linke tıklayarak chatbot arayüzüne erişin.
2.  Arayüz, temalı yapısı sayesinde kullanıcı dostu bir deneyim sunar.
3.  Projenin kabiliyetlerini test etmek için aşağıdaki soruları kullanabilirsiniz:
    * **Latte Tarifi:** "Latte nasıl yapılır? Adım adım anlat."
    * **Malzeme Sorgulama:** "Affogato için gereken malzemeler nelerdir?"
    * **Karşılaştırma:** "Dibek kahvesi ile normal Türk kahvesi arasındaki fark nedir?"
    * **Dış Bilgi Sınama:** "Su ısıtıcısı ne zaman icat edildi?" (Bu soruya "bilgi kaynaklarımda bulamadım" yanıtını vermesi beklenir.)